In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import os

# Set up seaborn
sns.set(style='whitegrid')

# --------------------------
# Load and preprocess data
# --------------------------
df = pd.read_csv('../data/processed_data/CREMP_Pcover_2023_TaxaGroups.csv', parse_dates=['Date'])
df_mean = df.groupby('Date').mean(numeric_only=True).reset_index()

# Define taxa columns
taxa_groups = [
    'Stony_coral', 'Macroalgae', 'Octocoral', 'Porifera',
    'Cyanobacteria', 'Seagrass', 'Zoanthidea', 'Urchins', 'Others'
]

# --------------------------
# Function to create lagged features
# --------------------------
def create_lagged_features(data, column, lags=5):
    df_lag = data[['Date', column]].copy()
    for i in range(1, lags+1):
        df_lag[f'lag_{i}'] = df_lag[column].shift(i)
    df_lag = df_lag.dropna().reset_index(drop=True)
    return df_lag

# --------------------------
# Create folder for plots
# --------------------------
plot_dir = '../forecasts'
os.makedirs(plot_dir, exist_ok=True)

# --------------------------
# Forecast storage
# --------------------------
forecast_all = []

# --------------------------
# Loop through taxa groups
# --------------------------
for taxa in taxa_groups:
    print(f"📊 Modeling and plotting: {taxa}")

    df_lagged = create_lagged_features(df_mean, taxa, lags=5)
    X = df_lagged[[f'lag_{i}' for i in range(1, 6)]]
    y = df_lagged[taxa]
    dates = df_lagged['Date']

    # Train/test split
    X_train, y_train = X[:-5], y[:-5]

    # Train models
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    xgb_model = XGBRegressor(n_estimators=100, random_state=42)
    xgb_model.fit(X_train, y_train)

    # Predict next 5 years
    rf_preds, xgb_preds = [], []
    last_input_rf = X.iloc[-1].values.reshape(1, -1)
    last_input_xgb = X.iloc[-1].values.reshape(1, -1)

    for _ in range(5):
        pred_rf = rf_model.predict(last_input_rf)[0]
        pred_xgb = xgb_model.predict(last_input_xgb)[0]
        rf_preds.append(pred_rf)
        xgb_preds.append(pred_xgb)
        last_input_rf = np.roll(last_input_rf, -1)
        last_input_rf[0, -1] = pred_rf
        last_input_xgb = np.roll(last_input_xgb, -1)
        last_input_xgb[0, -1] = pred_xgb

    # Build future date range
    future_dates = pd.date_range(start=df_mean['Date'].iloc[-1], periods=6, freq='Y')[1:]

    # Save forecasts
    for i in range(5):
        forecast_all.append({
            'year': future_dates[i].year,
            'taxa': taxa,
            'model': 'RandomForest',
            'forecast': rf_preds[i]
        })
        forecast_all.append({
            'year': future_dates[i].year,
            'taxa': taxa,
            'model': 'XGBoost',
            'forecast': xgb_preds[i]
        })

    # --------------------------
    # Plot historical + forecasts
    # --------------------------
    plt.figure(figsize=(10, 6))
    plt.plot(df_mean['Date'], df_mean[taxa], label='Observed', color='black')
    plt.plot(future_dates, rf_preds, '--o', label='Random Forest Forecast', color='blue')
    plt.plot(future_dates, xgb_preds, '--o', label='XGBoost Forecast', color='green')
    plt.title(f"{taxa.replace('_', ' ').title()} Forecast (5-Year)")
    plt.xlabel("Year")
    plt.ylabel("Percent Cover")
    plt.legend()
    plt.tight_layout()

    # Save plot
    plot_path = os.path.join(plot_dir, f"{taxa}_forecast.png")
    plt.savefig(plot_path)
    plt.close()

# --------------------------
# Save forecast CSV
# --------------------------
forecast_df = pd.DataFrame(forecast_all)
forecast_df.to_csv('../forecasts/coral_taxa_forecasts_regression_models.csv', index=False)

print(f"\n✅ Forecast CSV saved.")
print(f"📁 Plots saved to: {plot_dir}")


📊 Modeling and plotting: Stony_coral


c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegre

📊 Modeling and plotting: Macroalgae


c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegre

📊 Modeling and plotting: Octocoral


...

📊 Modeling and plotting: Urchins


c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegre

📊 Modeling and plotting: Others


c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Elite\Documents\Ocean\Conch Republic Coral Quest\coral-reef-trends\virtual_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegre


✅ Forecast CSV saved.
📁 Plots saved to: ../forecasts
